In [2]:
import digitalhub as dh
PROJECT_NAME = "landslide-monitoring"
proj = dh.get_or_create_project(PROJECT_NAME)

Register to the open data space copenicus(if not already) and get your credentials.

https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=FIiRPJeoiX4

Log the credentials as project secret keys as shown below

In [2]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

### Download data from Sentinel 1 (Orbit Direction Ascending)

In [2]:
function_s1 = proj.new_function("download_images_s1",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.11.5",command="python")

In [28]:
string_dict_data = """{
  'satelliteParams':{
      'satelliteType': 'Sentinel1',
      'processingLevel': 'LEVEL1',
      'sensorMode': 'IW',
      'productType': 'SLC',
      'orbitDirection': 'ASCENDING',
      'relativeOrbitNumber': '117'
  } ,
  'startDate': '2021-03-04',
  'endDate': '2021-03-10',
  'geometry': 'POLYGON((10.81295 45.895743, 10.813637 45.895743, 10.813637 45.89634, 10.81295 45.89634, 10.81295 45.895743))',
  'area_sampling': 'True',
  'tmp_path_same_folder_dwl':'True',
  'artifact_name': 's1_ascending'
  }"""
list_args =  ["main.py",string_dict_data]

In [ ]:
run = function_s1.run(
    action="job",
    secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
    fs_group='8877',
    args=["main.py", string_dict_data],
    resources={"cpu": {"requests": "3", "limits": "6"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-land",
        "mount_path": "/app/data",
        "spec": {
             "size": "300Gi" 
        }
    }])

### Download data from Sentinel 1 (Orbit Direction Descending)

In [ ]:
string_dict_data = """{
  'satelliteParams':{
      'satelliteType': 'Sentinel1',
      'processingLevel': 'LEVEL1',
      'sensorMode': 'IW',
      'productType': 'SLC',
      'orbitDirection': 'DESCENDING',
      'relativeOrbitNumber': '168'
  } ,
  'startDate': '2021-03-01',
  'endDate': '2021-03-15',
  'geometry': 'POLYGON((10.81295 45.895743, 10.813637 45.895743, 10.813637 45.89634, 10.81295 45.89634, 10.81295 45.895743))',
  'area_sampling': 'True',
  'tmp_path_same_folder_dwl':'True',
  'artifact_name': 's1_descending'
  }"""
list_args =  ["main.py",string_dict_data]

In [ ]:
run = function_s1.run(
    action="job",
    secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
    fs_group='8877',
    args=["main.py", string_dict_data],
    #resources={"cpu": {"requests": "3", "limits": "6"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-land",
        "mount_path": "/app/files",
        "spec": {
             "size": "300Gi" 
        }
    }])

### Perform Data Elaboration on RS-Tool

In [ ]:
# artifact_name='Shapes_TN'
# src_path='data/Shapes_TN'
# artifact_data = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

In [ ]:
function_rs = proj.new_function("elaborate",kind="container", image="ghcr.io/tn-aixpa/rs-landslide-monitoring:2.7_b9", command="/bin/bash", code_src="launch.sh")

In [22]:
#function_rs = proj.get_function("elaborate")

In [ ]:
run_el = function_rs.run(
    action="job",
    fs_group='8877',
    resources={"cpu": {"requests": "16", "limits": "24"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-land",
        "mount_path": "/app/files",
        "spec": { "size": "600Gi" }
    }],
    args=['/shared/launch.sh', 's1_ascending', 's1_descending', '2021-03-01', '2021-07-30', 'landslide_2021-03-04_2021-07-30', 'Shapes_TN', 'ammprv_v.shp','POLYGON ((11.687737 46.134408, 11.773911 46.134408, 11.773911 46.174363, 11.687737 46.174363, 11.687737 46.134408))']
)

In [7]:
run_el.refresh().status.state

'RUNNING'

### Pipeline

In [ ]:
%%writefile "landslide_pipeline.py"

from digitalhub_runtime_kfp.dsl import pipeline_context

def myhandler(startDate, endDate, geometry, outputName):
    with pipeline_context() as pc:
        string_dict_data_asc = """{"satelliteParams":{"satelliteType": "Sentinel1","processingLevel": "LEVEL1","sensorMode": "IW","productType": "SLC","orbitDirection": "ASCENDING","relativeOrbitNumber": "117"},"startDate": \"""" + str(startDate) + """\","endDate": \"""" + str(endDate) + """\","geometry": \"""" + str(geometry) + """\","area_sampling": "True","artifact_name": "s1_ascending"}"""
        string_dict_data_des = """{"satelliteParams":{"satelliteType": "Sentinel1","processingLevel": "LEVEL1","sensorMode": "IW","productType": "SLC","orbitDirection": "DESCENDING","relativeOrbitNumber": "168"},"startDate": \"""" + str(startDate) + """\","endDate": \"""" + str(endDate) + """\","geometry": \"""" + str(geometry) + """\","area_sampling": "True","artifact_name": "s1_descending"}"""
        
        s1 = pc.step(name="download-asc",
                     function="download_images_s1",
                     action="job",
                     secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
                     fs_group='8877',
                     args=["main.py", string_dict_data_asc],
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-land",
                        "mount_path": "/app/files",
                        "spec": { "size": "300Gi" }
                        }
                    ])

        s2 = pc.step(name="download-desc",
                     function="download_images_s1",
                     action="job",
                     secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
                     fs_group='8877',
                     args=["main.py", string_dict_data_des],
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-land",
                        "mount_path": "/app/files",
                        "spec": { "size": "300Gi" }
                        }
                    ]).after(s1)
        
        s3 = pc.step(name="elaborate",
                     function="elaborate",
                     action="job",
                     fs_group='8877',
                     resources={"cpu": {"requests": "6", "limits": "12"},"mem":{"requests": "32Gi", "limits": "64Gi"}},
                     volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-land",
                        "mount_path": "/app/data",
                        "spec": { "size": "2048Gi" }
                    }],
                     args=['/shared/launch.sh', 's1_ascending', 's1_descending', str(startDate), str(endDate), str(outputName), 'Shapes_TN', 'ammprv_v.shp', str(geometry)]
                     ).after(s2)
     


Overwriting landslide_pipeline.py


In [34]:
workflow = proj.new_workflow(name="pipeline_landslide", kind="kfp", code_src= "landslide_pipeline.py", handler = "myhandler")

In [35]:
wfbuild = workflow.run(action="build", wait=True)

2025-06-17 10:19:56,043 - INFO - Waiting for run e171287f8c5b438782b43460db98cc07 to finish...
2025-06-17 10:20:01,063 - INFO - Waiting for run e171287f8c5b438782b43460db98cc07 to finish...
2025-06-17 10:20:06,087 - INFO - Waiting for run e171287f8c5b438782b43460db98cc07 to finish...
2025-06-17 10:20:11,110 - INFO - Waiting for run e171287f8c5b438782b43460db98cc07 to finish...
2025-06-17 10:20:16,132 - INFO - Waiting for run e171287f8c5b438782b43460db98cc07 to finish...
2025-06-17 10:20:21,155 - INFO - Waiting for run e171287f8c5b438782b43460db98cc07 to finish...
2025-06-17 10:20:26,179 - INFO - Waiting for run e171287f8c5b438782b43460db98cc07 to finish...
2025-06-17 10:20:31,202 - INFO - Waiting for run e171287f8c5b438782b43460db98cc07 to finish...
2025-06-17 10:20:36,225 - INFO - Waiting for run e171287f8c5b438782b43460db98cc07 to finish...
2025-06-17 10:20:41,248 - INFO - Waiting for run e171287f8c5b438782b43460db98cc07 to finish...
2025-06-17 10:20:46,271 - INFO - Waiting for run e

In [36]:
workflow_run = workflow.run(action="pipeline", parameters={
    "startDate": "2021-03-01",
    "endDate": "2021-07-30",
    "geometry": "POLYGON((10.81295 45.895743, 10.813637 45.895743, 10.813637 45.89634, 10.81295 45.89634, 10.81295 45.895743))",
    "outputName": "landslide_2021-03_2021-07"
    })